In [9]:

%load_ext autoreload 
# %aimport rl_envs.grid_world_env

%autoreload 2
import math

from agents.approx_Q_Learning import ApproxQLearningAgent
from tools.helper import *
import  gymnasium  as gym
from rl_envs.new_gym_grid_world_env import GridWorldEnv
from collections import defaultdict
from torch.utils.tensorboard import SummaryWriter # type: ignore
import itertools
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
DISCOUNTED_FACTOR = 0.999
FORBIDDEN_REWARD = -10
HITWALL_REWARD = -10
TARGET_REWARD = 20
NORMAL_REWARD = -1

SEED = 666

In [11]:
# env = GridWorldEnv(
#     size=3,
#     fixed_map=True,
#     forbidden_grids=[(1, 1), (1, 2)],
#     target_grids=[(2, 2)],
#     forbidden_reward=FORBIDDEN_REWARD,
#     hit_wall_reward=HITWALL_REWARD,
#     target_reward=TARGET_REWARD,
#     normal_reward=NORMAL_REWARD,
# )
env = GridWorldEnv(fixed_map = True, forbidden_grids=[(1,1),(1,2), (2,2),(3,1),(3,3),(4,1)], target_grids=[(3,2)], forbidden_reward=FORBIDDEN_REWARD, hit_wall_reward=HITWALL_REWARD, target_reward=TARGET_REWARD, normal_reward=NORMAL_REWARD)


In [12]:
agent = ApproxQLearningAgent(action_space_n=env.action_n)

agent.RUN(env, num_episodes=3000, episode_len=2000)
# agent.RUN(env, num_episodes=10000, episode_len=2000)
# num_episodes=1000, episode_len=2000 能解出 3x3

Episode 0/3000 ( TD_error: -5.331814690172738)
Episode 10/3000 ( TD_error: 1.165263398306144)
Episode 20/3000 ( TD_error: 1.6800952451901257)
Episode 30/3000 ( TD_error: -0.29444528383147617)
Episode 40/3000 ( TD_error: -1.0262470456299027)
Episode 50/3000 ( TD_error: -0.7448001881171589)
Episode 60/3000 ( TD_error: -0.21804115588764628)
Episode 70/3000 ( TD_error: 0.06548388808657668)
Episode 80/3000 ( TD_error: 0.013808011344646709)
Episode 90/3000 ( TD_error: 0.1062745497868649)
Episode 100/3000 ( TD_error: 0.5146918652829378)
Episode 110/3000 ( TD_error: -0.034863099054389224)
Episode 120/3000 ( TD_error: -1.8212870231499547)
Episode 130/3000 ( TD_error: 0.32525406759486986)
Episode 140/3000 ( TD_error: 0.04657763168983564)
Episode 150/3000 ( TD_error: 0.004335038549239556)
Episode 160/3000 ( TD_error: 0.16716189907080592)
Episode 170/3000 ( TD_error: -0.020531351315086876)
Episode 180/3000 ( TD_error: 0.01162090607118138)
Episode 190/3000 ( TD_error: -0.16022346281039468)
Episode 

In [15]:
for y in range(env.height):
    for x in range(env.width):
        agent.policy_improvement((y, x))

Q = defaultdict(lambda: [0 for _ in range(agent.action_space_n)])
for y in range(env.height):
    for x in range(env.width):
        for action in range(agent.action_space_n):
            state = (y,x)
            Q[state][action] = agent.estimate_q(state, action)
print_by_dict(env, Q)

# ISSUE: 在 X 相同时, 不同的 y, q 值保持一致 (为何)


[ [24.461853981534954, 18.94435371723725, 12.018114490350628, 12.03240097449268] [19.75456721734441, 22.150853024538844, 9.94937571279461, 21.043009243529863] [25.672190421395552, 24.50413864330079, 13.142384568278416, 19.017167254335256] [28.29435449039401, 20.644467768202155, 15.458181532759133, 22.094061380156116] [24.053334627312175, 11.606191963605243, 11.656421158407033, 24.11026964366268] ]
[ [28.352017523319233, 19.676994366585795, 21.015654423435585, 15.534287388533] [33.003628408719315, 24.942124488801014, 18.92099828643447, 24.544727239551126] [38.85549249210844, 28.57116014217669, 22.12642797903855, 19.828114977057748] [32.877341222904185, 24.586202310805476, 24.431916857519454, 24.952637705325998] [28.45578045761405, 15.58254048126443, 20.63903991943219, 28.441686929691112] ]
[ [31.540645997815698, 32.92196292652968, 24.561126770579936, 19.680004598732303] [35.108340011984005, 37.86933989708695, 19.677259552404344, 28.699558512089535] [53.46072628669132, 33.19211917473968,

In [16]:
print_by_dict(env, agent.policy)
print()
print(agent.parameters)

print(env)
for i in range(env.height):
    print("[", end=" ")
    for j in range(env.width):
        state = (i, j)
        action = agent.get_action(state, optimal=True)
        print(env.action_mappings[action], end=" ")
    print("]")

[ [0.925 0.025 0.025 0.025] [0.025 0.925 0.025 0.025] [0.925 0.025 0.025 0.025] [0.925 0.025 0.025 0.025] [0.025 0.025 0.025 0.925] ]
[ [0.925 0.025 0.025 0.025] [0.925 0.025 0.025 0.025] [0.925 0.025 0.025 0.025] [0.925 0.025 0.025 0.025] [0.925 0.025 0.025 0.025] ]
[ [0.025 0.925 0.025 0.025] [0.025 0.925 0.025 0.025] [0.925 0.025 0.025 0.025] [0.925 0.025 0.025 0.025] [0.925 0.025 0.025 0.025] ]
[ [0.025 0.925 0.025 0.025] [0.025 0.925 0.025 0.025] [0.925 0.025 0.025 0.025] [0.025 0.025 0.025 0.925] [0.025 0.025 0.025 0.925] ]
[ [0.025 0.025 0.925 0.025] [0.025 0.925 0.025 0.025] [0.025 0.025 0.925 0.025] [0.025 0.025 0.025 0.925] [0.025 0.025 0.025 0.925] ]

[ 29.34775705   1.54118632  -0.36471574   1.88775334   2.05295483
  -1.32856175   3.16629136  -4.51009709  -1.11741962   1.8569358
  -5.56102585   0.2863968    1.35925983  -3.3355054    0.9310583
  -2.81016584   0.9408177   -0.11246243   0.19502917   0.46154174
   4.22828902   0.34080165  -2.68778178   1.53050824   0.05727921
 